In [1]:
# =====================================================
# DAY 4 – FEATURE ENGINEERING THAT WINS OFFERS (4 DEC 2025)
# Turn good model → unbeatable model
# =====================================================

In [3]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv(r"D:\Data Switch Journy\WA_Fn-UseC_-Telco-Customer-Churn.csv")
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce').fillna(0)

# Some TotalCharges values are empty strings → convert them to numbers
#  errors='coerce' --> turns invalid numbers into NaN
#.fillna(0) --> replaces NaN with 0
# This avoids errors in mathematical operations.

In [13]:
# Create a new feature-engineering dataframe
fe = pd.DataFrame()

In [15]:
# Basic Numeric Columns
fe['tenure'] = df['tenure']
fe['MonthlyCharges'] = df['MonthlyCharges']
fe['TotalCharges'] = df['TotalCharges']

In [17]:
# Feature 1 – Tenure_per_Charge

fe['Tenure_per_charges'] = df['tenure'] / (df['MonthlyCharges'] + 1)

# “What is the value per unit cost?”
#    -- Higher value → more loyal → less churn
#    -- Low tenure + high cost → churn risk
# +1 avoids division by zero.


In [19]:
# Feature 2 – Charges_Ratio

fe['Charges_Ratio'] = df['TotalCharges'] / (df['MonthlyCharges'] * df['tenure'] + 1)

# “How consistent are their payments relative to what they should have paid?”
#     --- If ratio < 1 → inconsistent usage → churn risk
#     --- If ratio > 1 → long-term consistent users
# Again +1 prevents division by zero.

In [21]:
# Feature 3 – HighValue

fe['HighValue'] = (df['tenure'] > 60) & (df['MonthlyCharges'] > 90)

# Boolean feature:
#   -- Long tenure + expensive plan = premium stable customer
#   -- Usually low churn probability

In [23]:
# Feature 4 – NewCustomer_Risk

fe['NewCustomer_Risk'] = (df['tenure'] <= 6) & (df['MonthlyCharges'] > 80)

# New customers who were put on expensive plans → churn often.
# This captures early dissatisfaction.

In [25]:
# Feature 5 – Contract_Tenure_Mismatch

fe['Contract_Tenure_Mismatch'] = df['Contract'].map({'Month-to-month':3, 'One year':2, 
                                                     'Two year':1}) * df['tenure']

#This is clever:
#  -- Month-to-month gets a higher multiplier (3) because those customers churn often.
#  -- Long contracts get lower weight (1).
#  -- Multiplying by tenure gives a “commitment × loyalty” composite.
# Higher mismatch = churn risk.

In [33]:
# Feature 6 – Service_Combo_Score (final stable fix)

service_cols = [
    'PhoneService','InternetService','OnlineSecurity',
    'TechSupport','StreamingTV','StreamingMovies'
]

# Step 1: Copy only needed columns
services = df[service_cols].copy()

# Step 2: Fix InternetService separately
services['InternetService'] = services['InternetService'].replace({
    'DSL': 1,
    'Fiber optic': 1,
    'No': 0
})

# Step 3: Replace Yes/No columns
services = services.replace({'Yes': 1, 'No': 0, 'No internet service': 0})

# Step 4: Convert all to integers (now safe)
services = services.astype(int)

# Step 5: Sum them up
fe['Service_Combo_Score'] = services.sum(axis=1)


C:\Users\hp\AppData\Local\Temp\ipykernel_22172\2062530815.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  services['InternetService'] = services['InternetService'].replace({
C:\Users\hp\AppData\Local\Temp\ipykernel_22172\2062530815.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  services = services.replace({'Yes': 1, 'No': 0, 'No internet service': 0})


In [35]:
# Feature 7 – Payment_Risk

fe['Payment_Risk'] = df['PaymentMethod'].map({
    'Electronic check':3, 
    'Mailed check':2, 
    'Bank transfer (automatic)':1, 
    'Credit card (automatic)':1
})

# Electronic check is famously the strongest churn predictor in Telco datasets.
# This encodes:
#   -- 3 = highest churn risk
#   -- 1 = lowest (auto payments

In [37]:
# Feature 8 – Target label

fe['Target'] = (df['Churn'] == 'Yes').astype(int)

# Binary encoding:
#  -- 1 = churn
#  -- 0 = not churn

In [41]:
# NEW FEATURE 09 - Tenure Stability Score

fe['Tenure_Stability'] = df['TotalCharges'] / (df['tenure'] + 1)

# Captures smoothness of customer spending:

#                            TotalCharges
#   Tenure_Stability  = ----------------------
#                             tenure + 1

# High = consistent long-term customer
# Low = new customer or scattered payments → higher churn

In [43]:
# Feature 12 — Support_Dependency

fe['Support_Dependency'] = (
    df[['OnlineSecurity', 'TechSupport']]
    .replace({'Yes':1, 'No':0, 'No internet service':0})
    .sum(axis=1)
)

# Measures reliance on “support-heavy” services:
#     -- (OnlineSecurity + TechSupport)

# Customers who need support but don't get satisfaction churn the most.

C:\Users\hp\AppData\Local\Temp\ipykernel_22172\3403310409.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({'Yes':1, 'No':0, 'No internet service':0})


In [45]:
fe['Target'] = (df['Churn'] == 'Yes').astype(int)

In [47]:
print("Top features by correlation with churn:")
print(fe.corr()['Target'].abs().sort_values(ascending=False).head(12))

Top features by correlation with churn:
Target                      1.000000
tenure                      0.352229
Tenure_per_charges          0.338704
Payment_Risk                0.282708
NewCustomer_Risk            0.220115
Support_Dependency          0.204042
TotalCharges                0.198324
MonthlyCharges              0.193356
Contract_Tenure_Mismatch    0.187615
HighValue                   0.097932
Tenure_Stability            0.072579
Charges_Ratio               0.015968
Name: Target, dtype: float64
